# Imports nécessaires

In [4]:
import pandas as pd
from datetime import datetime

# Classe qui a pour but de retourner efficacement toutes les destinations d'une ville

In [5]:
SEUIL_DISTANCE = 0.03

class DestiOptis:
    arrets_depart = []
    stops = []
    calendar_dates = []
    routes = []
    stop_times = []
    trips = []
    lat = 0
    lon = 0
    date_min = pd.to_datetime('2024-07-01')
    date_max = pd.to_datetime('2024-08-31')

    def __init__(self, path ='TER'):
        self.calendar_dates = pd.read_csv('Data/'+ path +'/calendar_dates.txt')
        self.stop_times = pd.read_csv('Data/'+ path +'/stop_times.txt')[['trip_id', 'stop_id', 'departure_time']]
        self.stops = pd.read_csv('Data/'+ path +'/stops.txt')[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station']]
        self.trips = pd.read_csv('Data/'+ path +'/trips.txt')[['service_id', 'trip_id', 'route_id']]
        self.calendar_dates.date = pd.to_datetime(self.calendar_dates["date"], format='%Y%m%d')
        self.stop_times['departure_time'] = pd.to_timedelta(self.stop_times['departure_time'])
        self.stops['parent_station'] = self.stops['parent_station'].fillna('')
    
    def load_search_destinations(self, lat, lon ,date_min= datetime.today,date_max = datetime.today):
        self.lat = lat
        self.lon = lon
        self.date_min = datetime.strptime(date_min, '%Y%m%d')
        self.date_max = datetime.strptime(date_max, '%Y%m%d')
    

    # Retourne les StopPoints proche du point de départ
    def stops_proches(self,lat,long): # Stop ID pour Global, parent_station pour SNCF
        return self.stops[(self.stops['stop_lat'] > lat-SEUIL_DISTANCE) & (self.stops['stop_lat'] < lat+SEUIL_DISTANCE) & (self.stops['stop_lon'] > long-SEUIL_DISTANCE) & (self.stops['stop_lon'] < long+SEUIL_DISTANCE) & self.stops['parent_station'].str.contains('StopArea')]
    
    def trips_stops_proches(self, lat, long):
        stops_proches = self.stops_proches(lat,long)
        trips = self.stop_times[self.stop_times['stop_id'].isin(stops_proches['stop_id'])]
        self.arrets_depart = trips.drop_duplicates(subset='trip_id')
        trips_ids = self.arrets_depart['trip_id']
        return trips_ids
    
    def trips_dans_periode(self, lat, long):
        trips_ids = self.trips_stops_proches(lat,long)
        trips = self.trips[self.trips['trip_id'].isin(trips_ids)]
        services = self.calendar_dates[self.calendar_dates['service_id'].isin(trips['service_id'])]
        services_dans_periode = services[(services['date'] > self.date_min) & (services['date'] < self.date_max)]
        services_dans_periode.drop_duplicates(subset='service_id')
        trips_dans_periode = trips[trips['service_id'].isin(services_dans_periode['service_id'])]
        return trips_dans_periode['trip_id']
    
    def get_destinations(self, lat, long):
        trips_ids_dans_periode = self.trips_dans_periode(lat,long)
        stop_times_arret_correct = self.stop_times[self.stop_times['trip_id'].isin(trips_ids_dans_periode)]
        stop_times_temps_superieur = stop_times_arret_correct.assign(temps_depart = "", stop_id_ville = "")

        stop_ids_depart = self.arrets_depart['stop_id']
        stop_ids_depart.index = self.arrets_depart['trip_id']
        stop_id_depart = stop_ids_depart.loc[stop_times_temps_superieur['trip_id']]
        stop_times_temps_superieur['stop_id_ville'] = stop_id_depart.array

        temps_depart = self.arrets_depart['departure_time'] #StopTime trip
        temps_depart.index = self.arrets_depart['trip_id']
        colonne_temps_depart = temps_depart.loc[stop_times_temps_superieur['trip_id']]
        stop_times_temps_superieur['temps_ville'] = colonne_temps_depart.array

        destinations_doublons_stop_points = stop_times_temps_superieur[stop_times_temps_superieur['departure_time'] > stop_times_temps_superieur['temps_ville']]
        destinations_doublons_stop_points = self.stops[self.stops['stop_id'].isin(destinations_doublons_stop_points['stop_id'])]
        destinations_doublons_stop_area = self.stops[self.stops['stop_id'].isin(destinations_doublons_stop_points['parent_station'])]
        destinations_stop_area = destinations_doublons_stop_area.drop_duplicates(subset='stop_id')
        destinations = destinations_stop_area[~destinations_stop_area['stop_id'].isin(self.stops_proches(lat,long)['parent_station'])]
        return destinations
    

In [6]:
# Initialisation pour les tests
desti = DestiOptis()
import AnalyzerGTFS as ag

In [8]:
# Coordonnées d'Orléans : 47.9078910, 1.90424200
import time

AnalyzerGTFS = ag.AnalyzerGTFS()
st1 = time.time()
a = AnalyzerGTFS.get_destinations(47.9078910, 1.90424200,'20240718', '20240718')
et1 = time.time()

st2 = time.time()
a = desti.get_destinations(47.9078910, 1.90424200)
et2 = time.time()

print(et1-st1)
print(et2-st2)


#print(desti.trips_dans_periode_2(47.9078910, 1.90424200))
#print(len(desti.trips_dans_periode(47.9078910, 1.90424200)))
#print(len(desti.trips_dans_periode_2(47.9078910, 1.90424200)))
#print(desti.trips_proches(47.9078910, 1.90424200))

                   stop_id                      stop_name   stop_lat  \
3807  StopArea:OCE87481002                         Nantes  47.217491   
3828  StopArea:OCE87481192                        Ancenis  47.369334   
3910  StopArea:OCE87484006              Angers Saint-Laud  47.464649   
4066  StopArea:OCE87487603                         Saumur  47.268994   
4223  StopArea:OCE87543017                    Les Aubrais  47.926801   
...                    ...                            ...        ...   
5180  StopArea:OCE87597070                 Neuvy-Pailloux  46.882716   
5189  StopArea:OCE87597120            Argenton-sur-Creuse  46.592250   
5193  StopArea:OCE87597153                         Éguzon  46.439769   
6549  StopArea:OCE87696005                         Nevers  46.987282   
7512  StopArea:OCE87726364  Chartres Pôle d'Échange Compa  48.446547   

      stop_lon parent_station  
3807 -1.542077                 
3828 -1.177763                 
3910 -0.556770                 
4066 -0

In [ ]:
a = desti.trips_dans_periode(47.9078910, 1.90424200)



# Optimisation de la recherche de trajet entre deux villes

In [ ]:
SEUIL_DISTANCE = 0.03
import pandas as pd
from datetime import datetime

class TrajetOptis:
    arrets_depart = []
    stops = []
    calendar_dates = []
    routes = []
    stop_times = []
    trips = []
    lat = 0
    lon = 0
    date_min = pd.to_datetime('2024-07-01')
    date_max = pd.to_datetime('2024-08-31')

    def __init__(self, path ='TER'):
        self.calendar_dates = pd.read_csv('Data/'+ path +'/calendar_dates.txt')
        self.stop_times = pd.read_csv('Data/'+ path +'/stop_times.txt')[['trip_id', 'stop_id', 'departure_time']]
        self.stops = pd.read_csv('Data/'+ path +'/stops.txt')[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station']]
        self.trips = pd.read_csv('Data/'+ path +'/trips.txt')[['service_id', 'trip_id', 'route_id']]
        self.calendar_dates.date = pd.to_datetime(self.calendar_dates["date"], format='%Y%m%d')
        self.stop_times['departure_time'] = pd.to_timedelta(self.stop_times['departure_time'])
        self.stops['parent_station'] = self.stops['parent_station'].fillna('')
    
    def load_search_destinations(self, lat, lon ,date_min= datetime.today,date_max = datetime.today):
        self.lat = lat
        self.lon = lon
        self.date_min = datetime.strptime(date_min, '%Y%m%d')
        self.date_max = datetime.strptime(date_max, '%Y%m%d')

    def get_trajets(self, lat_arrivee, lon_arrivee):
        stops_depart = self.stops[(self.stops['stop_lat'] > self.lat-SEUIL_DISTANCE) & (self.stops['stop_lat'] < self.lat+SEUIL_DISTANCE) & (self.stops['stop_lon'] > self.lon-SEUIL_DISTANCE) & (self.stops['stop_lon'] < self.lon+SEUIL_DISTANCE) & self.stops['parent_station'].str.contains('StopArea')]
        stops_arrivee = self.stops[(self.stops['stop_lat'] > lat_arrivee-SEUIL_DISTANCE/2) & (self.stops['stop_lat'] < lat_arrivee+SEUIL_DISTANCE/2) & (self.stops['stop_lon'] > lon_arrivee-SEUIL_DISTANCE/2) & (self.stops['stop_lon'] < lon_arrivee+SEUIL_DISTANCE/2) & self.stops['parent_station'].str.contains('StopArea')]
        trajets_avec_stops_depart = self.stop_times[self.stop_times['stop_id'].isin(stops_depart['stop_id'])]
        trajets_avec_stops_arrivee = self.stop_times[self.stop_times['stop_id'].isin(stops_arrivee['stop_id'])]
        trips_avec_depart_arrivee =  pd.merge(trajets_avec_stops_depart,trajets_avec_stops_arrivee, on = 'trip_id')
        trips_heure = trips_avec_depart_arrivee[trips_avec_depart_arrivee['departure_time_x'] < trips_avec_depart_arrivee['departure_time_y']]
        trips = pd.merge(trips_heure, self.trips, on = 'trip_id')
        trips_et_jours = pd.merge(trips, self.calendar_dates, on = 'service_id')
        trajets = trips_et_jours[(trips_et_jours['date'] >= self.date_min) & (trips_et_jours['date'] <= self.date_max)]
        trajets = trajets.assign(horaire_depart = "", horaire_arrivee = "")
        trajets['horaire_depart'] = trajets.apply(lambda x : x['departure_time_x']+x['date'], axis=1)
        trajets['horaire_arrivee'] = trajets.apply(lambda x : x['departure_time_y']+x['date'], axis=1)
        return trajets

    # Retourne les StopPoints proche du point de départ
    def stops_proches(self,lat,long): # Stop ID pour Global, parent_station pour SNCF
        return self.stops[(self.stops['stop_lat'] > lat-SEUIL_DISTANCE) & (self.stops['stop_lat'] < lat+SEUIL_DISTANCE) & (self.stops['stop_lon'] > long-SEUIL_DISTANCE) & (self.stops['stop_lon'] < long+SEUIL_DISTANCE) & self.stops['parent_station'].str.contains('StopArea')]
    

In [ ]:
trajet_Opti = TrajetOptis()

In [1]:
import AnalyzerGTFS
AnalyzerGTFS = AnalyzerGTFS.AnalyzerGTFS()

In [ ]:
import time

st1 = time.time()
a = AnalyzerGTFS.trajet_destination('StopArea:OCE87547000')
print(len(a))
et1 = time.time()

st2 = time.time()
a = trajet_Opti.get_trajets(48.8422850,2.36489100)
print(len(a))
et2 = time.time()

print(et1-st1)
print(et2-st2)
a = a.sort_values(by=['departure_time_x'])
for i in a.itertuples():
    print(i.horaire_depart, i.horaire_arrivee)